In [16]:
import os
import pandas as pd


from zc_combine.utils.naslib_utils import parse_scores, load_search_space, load_search_spaces_multiple
from zc_combine.ensemble.filter import common_n_largest, filter_by_zc
from zc_combine.utils.plot_utils import plot_common_networks, plot_accuracy_histogram, plot_networks_by_zc, plot_top_accuracy_zc, plot_filtered_by_zc, plot_filtered_ranks
from zc_combine.ensemble.eval import get_stats_zc, get_stats_ranks

naslib_path = '../zero_cost/NASLib'

all_spaces = [os.path.splitext(f)[0] for f in os.listdir(os.path.join(naslib_path, 'naslib/data/')) if '.json' in f]
all_spaces

['zc_transbench101_macro',
 'zc_transbench101_micro',
 'zc_nasbench301',
 'zc_nasbench201',
 'zc_nasbench101']

In [17]:
what = 'zc_nasbench201'

zc_bench = load_search_space(naslib_path, what)
dfs = parse_scores(zc_bench)

In [18]:
dfs['cifar10'].columns

Index(['net', 'val_accs', 'epe_nas', 'fisher', 'flops', 'grad_norm', 'grasp',
       'jacov', 'l2_norm', 'nwot', 'params', 'plain', 'snip', 'synflow', 'zen',
       'random'],
      dtype='object')

In [19]:
dataset = 'cifar10'
trio = ['synflow', 'snip', 'jacov']
n_vals = 200

data = dfs[dataset][[c for c in trio]].iloc[:n_vals]

In [20]:
data

,synflow,snip,jacov
0,44.484811,22.923178,-8.828987e+01
1,85.295247,26.610859,-6.519331e+01
2,51.838876,44.178249,-6.615395e+01
3,60.212156,51.889561,-6.528531e+01
4,0.000000,0.000000,-1.000000e+08
...,...,...,...
195,64.252077,15.870172,-6.516611e+01
196,71.502344,81.845146,-6.579324e+01
197,61.461608,38.061649,-6.848107e+01
198,58.040179,77.397430,-6.569703e+01


In [21]:
result = pd.DataFrame(columns=data.columns,
                      index=pd.MultiIndex.from_product([data.index, data.index]))
result

synflow snip jacov
0   0       NaN  NaN   NaN
    1       NaN  NaN   NaN
    2       NaN  NaN   NaN
    3       NaN  NaN   NaN
    4       NaN  NaN   NaN
...         ...  ...   ...
199 195     NaN  NaN   NaN
    196     NaN  NaN   NaN
    197     NaN  NaN   NaN
    198     NaN  NaN   NaN
    199     NaN  NaN   NaN

[40000 rows x 3 columns]

In [22]:
import numpy as np


for c in result.columns:
    result[c] = np.less_equal.outer(data[c].to_numpy(), data[c].to_numpy()).ravel()

result

synflow   snip  jacov
0   0       True   True   True
    1       True   True   True
    2       True   True   True
    3       True   True   True
    4      False  False  False
...          ...    ...    ...
199 195     True  False   True
    196     True   True   True
    197     True   True   True
    198     True   True   True
    199     True   True   True

[40000 rows x 3 columns]

In [23]:
result.astype(int)

synflow  snip  jacov
0   0          1     1      1
    1          1     1      1
    2          1     1      1
    3          1     1      1
    4          0     0      0
...          ...   ...    ...
199 195        1     0      1
    196        1     1      1
    197        1     1      1
    198        1     1      1
    199        1     1      1

[40000 rows x 3 columns]

In [24]:
result['comp'] = result.sum(axis=1)
result['comp'] = result['comp'] > 1
result['comp']

0    0       True
     1       True
     2       True
     3       True
     4      False
            ...  
199  195     True
     196     True
     197     True
     198     True
     199     True
Name: comp, Length: 40000, dtype: bool

In [25]:
edges = result[result['comp']]
edges.index

MultiIndex([(  0,   0),
            (  0,   1),
            (  0,   2),
            (  0,   3),
            (  0,   5),
            (  0,   7),
            (  0,   8),
            (  0,   9),
            (  0,  15),
            (  0,  16),
            ...
            (199, 188),
            (199, 189),
            (199, 190),
            (199, 193),
            (199, 194),
            (199, 195),
            (199, 196),
            (199, 197),
            (199, 198),
            (199, 199)],
           length=20125)

In [26]:
len(edges)

20125

In [27]:
edge_set = set(edges.index.to_list())

In [28]:
edge_dict = {}

for e0, e1 in edge_set:
    vals = edge_dict.setdefault(e0, [])
    vals.append(e1)

In [29]:
cycles = 0
visited = set()

for v1, vals in edge_dict.items():
    for v2 in vals:
        for v3 in edge_dict[v2]:
            key = tuple(sorted((v1, v2, v3)))
            if key in visited:
                continue

            visited.add(key)
            if (v3, v1) in edge_set:
                cycles += 1

In [30]:
cycles

13936

In [31]:
from math import comb

cycles / comb(n_vals, 3)

0.01061062890208619